SCRAPING

In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time
import pandas as pd
import mysql.connector
import re
from nltk.collocations import *
from nltk.corpus import stopwords

In [2]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    service = Service(executable_path=path)
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    driver.get(url)
    jobs = []
    job_buttons=[]
    time_show_more_clicked=0
    while len(job_buttons) < num_jobs:  #If true, should be still looking for new jobs.
        time.sleep(slp_time)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "JobsList_jobsList__lqjTr")))
        job_buttons = driver.find_elements(By.CLASS_NAME, "JobsList_jobListItem__wjTHv")
        time.sleep(5)
        try:
            driver.find_element(By.XPATH, "//button[span/span[text()='Show more jobs']]").click()
            time_show_more_clicked+=1
            time.sleep(5*time_show_more_clicked)
            print("See More Jobs clicked !")
            if len(job_buttons) == len(driver.find_elements(By.CLASS_NAME, "JobsList_jobListItem__wjTHv")):#if see more button doesnt works
                driver.refresh()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(job_buttons)))
            break
    
    print("Job cards loaded : ",len(job_buttons))

    for index,job_button in enumerate(job_buttons):
        print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs))) #shows progress
        if len(jobs) >= num_jobs:
            break
    
        job_button.click() #Clicks a job card
        time.sleep(2)
        collected_successfully = False
        attempts=0

        try:
            short_description_elements=driver.find_elements(By.CLASS_NAME, "JobCard_jobDescriptionSnippet__l1tnl")
            short_desc = short_description_elements[index].text
        except NoSuchElementException:
            short_desc= -1
            print("Can not read short description")
            
        while not collected_successfully and attempts < 5:
            try:
                driver.find_element(By.CLASS_NAME, 'ShowMoreCTA_showMore__EtZpZ').click()
                time.sleep(0.5)
                try:
                    job_elements = driver.find_elements(By.CSS_SELECTOR, 'li[data-jobid]')
                    job_id=job_elements[index].get_attribute('data-jobid')
                except:
                    print("can not read job_ID")
                try:
                    job_description = driver.find_element(By.CLASS_NAME, "JobDetails_jobDescription__uW_fK").text
                except:
                    print("Can not read job Description")
                    job_description=-1
                try:
                    title = driver.find_element(By.CLASS_NAME, "heading_Level1__soLZs").text
                except:
                    print("Can not read title")
                    title=-1
                try:
                    company=driver.find_element(By.CLASS_NAME, "heading_Subhead__Ip1aW").text
                except:
                    print("Can not read company")
                    company=-1
                try:
                    location_unprocessed=driver.find_element(By.CLASS_NAME, "JobDetails_locationAndPay__XGFmY").text
                    location = location_unprocessed.split('\n')[0]
                except:
                    print("Can not read company")
                    location=-1
                
                collected_successfully = True    
            except:
                print("Can not click see more button")
                time.sleep(1)
                attempts+=1
        
    
        #Printing for debugging
        if verbose:
            print("Job ID: {}".format(job_id))
            print("Short Desc: {}".format(short_desc))
            print("Job Title: {}".format(title))
            print("Company Name: {}".format(company))
            print("Location: {}".format(location))
            print("Job Description: {}".format(job_description))

        if short_desc != -1 and job_description != -1:
            combined_description = f"{short_desc}\n{job_description}"
        elif job_description == -1:
            combined_description = short_desc
        else:
            combined_description = job_description

        jobs.append({"job_id" : job_id,
        "job_title" : keyword,
        "company_name" : company,
        "job_description" : combined_description,
        "location" : location})

    return pd.DataFrame(jobs)

In [3]:

def keywords_extraction(df, lang_path, skill_path, tech_path, education_path):
    stop = set(stopwords.words('english'))
    cleaned_descriptions = []
    languages_found = []
    skills_found = []
    techs_found = []
    education_found = []
  
    # Load and preprocess keywords from all three CSVs
    def load_keywords(path, column_name):
        keywords_df = pd.read_csv(path)
        return [kw.lower() for kw in keywords_df[column_name].dropna().tolist()]

    language_list = load_keywords(lang_path, "Language")
    skill_list = load_keywords(skill_path, "Skill")
    tech_list = load_keywords(tech_path, "Technology")
    education_list = load_keywords(education_path, "Education")

    for desc in df["job_description"]:
        # Cleaning
        desc_clean = desc.replace("-", " ")
        desc_clean = re.sub(r"[\n]", " ", desc_clean)
        desc_clean = re.sub(r"[.!?/()\-,:;]", " ", desc_clean)
        desc_clean = re.sub(r"\d+[+-]?", " ", desc_clean)
        desc_clean = re.sub(r"[’']", " ", desc_clean)
        desc_clean = desc_clean.lower()
        desc_clean = " ".join([word for word in desc_clean.split() if word and word not in stop])
        cleaned_descriptions.append(desc_clean)

        # Keyword Matching (unique matches only)
        words = desc_clean.split()
        languages = []
        skills = []
        techs = []
        education = []
        
        for word in words:
            if word in language_list and word not in languages:
                languages.append(word)
            if word in skill_list and word not in skills:
                skills.append(word)
            if word in tech_list and word not in techs:
                techs.append(word)
            if word in education_list and word not in education:
                education.append(word)
        
        languages_found.append(", ".join(languages))
        skills_found.append(", ".join(skills))
        techs_found.append(", ".join(techs))
        education_found.append(", ".join(education))

    # Add columns to DataFrame
    df["Languages"] = languages_found
    df["Skills"] = skills_found
    df["Technologies"] = techs_found
    df["Education"] = education_found

    return df


In [4]:
def df_to_MySQL(df, user, password, host, database, table_name):
    try:
        # Connect to the database
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        
        # Create cursor
        cursor = conn.cursor()

        # Iterate through the DataFrame rows and insert data
        for index, row in df.iterrows():
            # Prepare the INSERT IGNORE query
            query = f"""
            INSERT IGNORE INTO job_data (job_id, job_title, location, Languages, Technologies, Skills, Education)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            
            # Execute the query for each row
            cursor.execute(query, (row['job_id'], row['job_title'], row['location'], row['Languages'], row['Technologies'], row['Skills'], row['Education']))
        
        # Commit the transaction
        conn.commit()

        print("Data inserted successfully!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        # Close the cursor and connection
        if conn.is_connected():
            cursor.close()
            conn.close()

In [5]:
title=["Data Scientist", "Data Analyst", "AI Engineer"]
table=["data_scientist", "data_analyst", "ai_engineer"]
print(title)
role=int(input("Select Job title: 1, 2, 3 :"))
num_of_jobs=int(input("Enter number of Jobs to Scrap: "))
path = "C:/Users/PMLS/Downloads/chromedriver.exe"
df_raw = get_jobs(keyword=title[role-1], num_jobs=num_of_jobs, verbose=False, path=path, slp_time=5)
df_with_keywords = keywords_extraction(df=df_raw, lang_path="C:/Users/PMLS/Downloads/languages.csv", skill_path="C:/Users/PMLS/Downloads/skills.csv", tech_path="C:/Users/PMLS/Downloads/technologies.csv", education_path="C:/Users/PMLS/Downloads/educations.csv")
df_to_MySQL(df=df_with_keywords, user="root", password="1234", host="localhost", database="job_data", table_name="job_data")
df_with_keywords.to_csv(f"glassdoor_jobs_{table[role-1]}.csv", index=False)
print("CSV file generated")

['Data Scientist', 'Data Analyst', 'AI Engineer']


Select Job title: 1, 2, 3 : 3
Enter number of Jobs to Scrap:  10


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 135
Current browser version is 137.0.7151.104 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF7055C5335+78597]
	GetHandleVerifier [0x00007FF7055C5390+78688]
	(No symbol) [0x00007FF7053791AA]
	(No symbol) [0x00007FF7053BD1CF]
	(No symbol) [0x00007FF7053BC21B]
	(No symbol) [0x00007FF7053B65A8]
	(No symbol) [0x00007FF7053B1A5D]
	(No symbol) [0x00007FF7054051E5]
	(No symbol) [0x00007FF7054047A0]
	(No symbol) [0x00007FF7053F6EC3]
	(No symbol) [0x00007FF7053C03F8]
	(No symbol) [0x00007FF7053C1163]
	GetHandleVerifier [0x00007FF70586EEED+2870973]
	GetHandleVerifier [0x00007FF705869698+2848360]
	GetHandleVerifier [0x00007FF705886973+2967875]
	GetHandleVerifier [0x00007FF7055E017A+188746]
	GetHandleVerifier [0x00007FF7055E845F+222255]
	GetHandleVerifier [0x00007FF7055CD2B4+111236]
	GetHandleVerifier [0x00007FF7055CD462